In [1]:
import json
import pandas as pd

In [2]:
with open("vns_benchmarks_300s.json", "r", encoding="utf-8") as f:
    vns_300s = json.load(f)

with open("gurobi_benchmarks_verbose.json", "r", encoding="utf-8") as f:
    gurobi_60s = json.load(f)

vns_300s: dict
gurobi_60s: dict

print(vns_300s.keys() == gurobi_60s.keys())

True


In [3]:
project_quantities = [3, 4, 5]
student_quantities = [30, 40, 50]


def get_benchmarking_data_by_dimension(benchmarking_data: dict):
    benchmarking_data_by_dimension = {}
    for project_quantity in project_quantities:
        benchmarking_data_by_dimension[project_quantity] = {}
        for student_quantity in student_quantities:
            benchmarking_data_by_dimension[project_quantity][student_quantity] = {}

    for instance, solutions in benchmarking_data.items():
        for project_quantity in project_quantities:
            for student_quantity in student_quantities:
                if f"_{project_quantity}_{student_quantity}" in instance:
                    benchmarking_data_by_dimension[project_quantity][student_quantity][int(instance[-1])] = solutions
    return benchmarking_data_by_dimension


vns_300s_dimensions = get_benchmarking_data_by_dimension(vns_300s)
gurobi_60s_dimensions = get_benchmarking_data_by_dimension(gurobi_60s)

In [4]:
def round_minor_deviations(x: float) -> float | int:
    nearest = round(x)
    if abs(x - nearest) <= 1e-6:
        return nearest
    return x


def best_solution_whithin_time_limit(solutions, time_limit=300):
    for solution in reversed(solutions):
        if solution["runtime"] <= time_limit:
            return solution


def runtime_best_objective_gurobi(solutions, best_objective):
    for solution in solutions:
        if round_minor_deviations(solution["incumbent_obj"]) == best_objective:
            return solution["runtime"]

In [5]:
results_dimensions = {}
for project_quantity in project_quantities:
    results_dimensions[project_quantity] = {}
    for student_quantity in student_quantities:
        vns_300s_dimension = vns_300s_dimensions[project_quantity][student_quantity]
        vns_300s_dimension: dict
        results_dimensions[project_quantity][student_quantity] = {}
        results_dimension = results_dimensions[project_quantity][student_quantity]
        results_dimension["granular"] = pd.DataFrame.from_dict(
            {
                instance: {
                    # "Instance": instance,
                    "Best VNS": (best_vns := int((solution := best_solution_whithin_time_limit(solutions))["obj"])),
                    "Runtime VNS (s)": float(solution["runtime"]),
                    "Best Gurobi": (
                        best_gurobi := round_minor_deviations(
                            (gurobi_solutions := gurobi_60s_dimensions[project_quantity][student_quantity][instance])[
                                -1
                            ]["incumbent_obj"]
                        )
                    ),
                    "Runtime Gurobi (s)": runtime_best_objective_gurobi(gurobi_solutions, best_gurobi),
                    "Upper Bound": (upper_bound_gurobi := round_minor_deviations(gurobi_solutions[-1]["obj_bound"])),
                    "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
                    "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
                    "Gap VNS": (upper_bound_gurobi - best_vns) / upper_bound_gurobi,
                }
                for instance, solutions in vns_300s_dimension.items()
            },
            orient="index",
        )

        results_dimension_granular = results_dimension["granular"]
        results_dimension_granular: pd.DataFrame

        means = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].mean()
        medians = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].median()
        maxima = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].max()
        minima = results_dimension_granular[
            ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
        ].min()
        results_dimension["summary"] = pd.DataFrame(
            [means, medians, maxima, minima], index=["Mean", "Median", "Maximum", "Minimum"]
        )
        results_dimension_summary = results_dimension["summary"]
        results_dimension_summary: pd.DataFrame

        results_dimension_summary_presentation = results_dimension_summary.copy()
        results_dimension_summary_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
            results_dimension_summary_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
        )
        results_dimension_summary_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = (
            results_dimension_summary_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]].map("{:.2f}".format)
        )
        results_dimension["summary_presentation"] = results_dimension_summary_presentation

        results_dimension_presentation = results_dimension_granular.copy()
        results_dimension_presentation[["Best VNS", "Best Gurobi", "Upper Bound"]] = results_dimension_presentation[
            ["Best VNS", "Best Gurobi", "Upper Bound"]
        ].map("{:.0f}".format)
        results_dimension_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
            results_dimension_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
        )
        results_dimension_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = results_dimension_presentation[
            ["Runtime VNS (s)", "Runtime Gurobi (s)"]
        ].map("{:.2f}".format)
        results_dimension_presentation.index.name = "Instance"
        results_dimension["granular_presentation"] = results_dimension_presentation

In [6]:
results_dimensions[3][30]["granular_presentation"]


,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,79,0.61,81,0.35,81,2.47%,0.00%,2.47%
1,92,3.38,92,2.76,92,0.00%,0.00%,0.00%
2,86,10.27,86,0.60,86,0.00%,0.00%,0.00%
3,95,34.69,98,0.95,102,3.06%,3.92%,6.86%
4,99,42.86,101,3.30,101,1.98%,0.00%,1.98%
5,96,8.16,96,0.49,96,0.00%,0.00%,0.00%
6,98,143.89,98,0.31,98,0.00%,0.00%,0.00%
7,69,7.44,71,11.32,75,2.82%,5.33%,8.00%
8,100,82.02,101,0.85,104,0.99%,2.88%,3.85%


In [7]:
results_dimensions[3][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,33.71,2.24,1.26%,2.11%,3.33%
Median,9.22,0.90,1.11%,0.00%,2.22%
Maximum,143.89,11.32,3.06%,8.99%,10.11%
Minimum,0.61,0.31,0.00%,0.00%,0.00%


In [8]:
results_dimensions[3][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,117,66.26,118,3.13,118,0.85%,0.00%,0.85%
1,105,78.76,112,0.92,119,6.25%,5.88%,11.76%
2,114,5.73,116,5.85,123,1.72%,5.69%,7.32%
3,113,9.19,114,4.31,114,0.88%,0.00%,0.88%
4,121,87.80,123,5.41,134,1.63%,8.21%,9.70%
5,115,2.27,122,10.34,130,5.74%,6.15%,11.54%
6,102,35.69,102,0.42,109,0.00%,6.42%,6.42%
7,103,129.34,105,2.05,115,1.90%,8.70%,10.43%
8,119,16.27,121,4.46,124,1.65%,2.42%,4.03%


In [9]:
results_dimensions[3][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,43.33,3.79,2.06%,4.52%,6.47%
Median,25.98,3.72,1.64%,5.79%,6.87%
Maximum,129.34,10.34,6.25%,8.70%,11.76%
Minimum,2.02,0.42,0.00%,0.00%,0.85%


In [10]:
results_dimensions[3][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,155,226.52,160,2.56,164,3.12%,2.44%,5.49%
1,151,72.09,154,1.28,159,1.95%,3.14%,5.03%
2,138,270.20,139,9.85,146,0.72%,4.79%,5.48%
3,120,22.61,123,2.03,133,2.44%,7.52%,9.77%
4,83,210.54,88,27.82,93,5.68%,5.38%,10.75%
5,144,269.06,147,26.98,159,2.04%,7.55%,9.43%
6,131,208.22,133,6.36,139,1.50%,4.32%,5.76%
7,150,42.10,154,1.91,165,2.60%,6.67%,9.09%
8,130,12.69,136,8.49,140,4.41%,2.86%,7.14%


In [11]:
results_dimensions[3][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,140.85,13.62,2.83%,5.24%,7.92%
Median,141.33,7.43,2.52%,5.09%,8.12%
Maximum,270.20,48.88,5.68%,7.75%,11.27%
Minimum,12.69,1.28,0.72%,2.44%,5.03%


In [12]:
results_dimensions[4][30]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,89,149.37,89,0.71,89,0.00%,0.00%,0.00%
1,92,54.04,96,0.95,96,4.17%,0.00%,4.17%
2,103,87.72,102,2.84,109,-0.98%,6.42%,5.50%
3,103,16.38,104,0.51,104,0.96%,0.00%,0.96%
4,106,1.13,111,4.85,116,4.50%,4.31%,8.62%
5,88,82.75,89,1.90,94,1.12%,5.32%,6.38%
6,111,259.40,114,4.89,114,2.63%,0.00%,2.63%
7,90,46.88,95,2.17,95,5.26%,0.00%,5.26%
8,84,1.89,87,0.67,87,3.45%,0.00%,3.45%


In [13]:
results_dimensions[4][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,70.06,2.06,2.53%,2.19%,4.68%
Median,50.46,1.52,3.04%,0.00%,4.71%
Maximum,259.40,4.89,5.26%,6.42%,9.80%
Minimum,1.06,0.51,-0.98%,0.00%,0.00%


In [14]:
results_dimensions[4][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,128,258.47,129,0.62,133,0.78%,3.01%,3.76%
1,115,13.38,118,0.88,128,2.54%,7.81%,10.16%
2,124,6.00,123,7.56,137,-0.81%,10.22%,9.49%
3,115,202.73,116,6.15,122,0.86%,4.92%,5.74%
4,113,103.85,116,0.43,116,2.59%,0.00%,2.59%
5,114,243.37,118,1.33,130,3.39%,9.23%,12.31%
6,118,121.02,121,12.59,127,2.48%,4.72%,7.09%
7,121,5.55,126,14.89,126,3.97%,0.00%,3.97%
8,130,74.44,135,10.34,139,3.70%,2.88%,6.47%


In [15]:
results_dimensions[4][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,110.73,10.72,2.17%,4.78%,6.87%
Median,91.14,6.86,2.51%,4.82%,6.78%
Maximum,258.47,52.42,3.97%,10.22%,12.31%
Minimum,5.55,0.43,-0.81%,0.00%,2.59%


In [16]:
results_dimensions[4][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,153,121.01,159,20.01,166,3.77%,4.22%,7.83%
1,144,20.34,148,6.21,159,2.70%,6.92%,9.43%
2,158,54.31,160,5.81,160,1.25%,0.00%,1.25%
3,145,196.18,150,5.32,160,3.33%,6.25%,9.38%
4,168,50.92,170,8.60,176,1.18%,3.41%,4.55%
5,147,14.05,153,2.34,153,3.92%,0.00%,3.92%
6,153,83.18,158,6.79,171,3.16%,7.60%,10.53%
7,153,166.22,156,5.80,161,1.92%,3.11%,4.97%
8,153,20.55,160,3.82,173,4.38%,7.51%,11.56%


In [17]:
results_dimensions[4][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,101.70,7.78,2.77%,4.42%,7.06%
Median,68.74,6.01,2.93%,4.72%,7.51%
Maximum,290.29,20.01,4.38%,7.60%,11.56%
Minimum,14.05,2.34,1.18%,0.00%,1.25%


In [18]:
results_dimensions[5][30]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,94,1.89,97,12.52,97,3.09%,0.00%,3.09%
1,89,102.87,94,1.17,101,5.32%,6.93%,11.88%
2,100,66.31,102,0.92,112,1.96%,8.93%,10.71%
3,107,42.12,110,5.03,115,2.73%,4.35%,6.96%
4,92,71.30,103,37.33,103,10.68%,0.00%,10.68%
5,98,184.87,100,12.35,100,2.00%,0.00%,2.00%
6,84,64.73,87,2.42,87,3.45%,0.00%,3.45%
7,92,150.69,94,8.46,101,2.13%,6.93%,8.91%
8,102,4.95,103,20.00,108,0.97%,4.63%,5.56%


In [19]:
results_dimensions[5][30]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,71.20,10.46,3.45%,3.18%,6.54%
Median,65.52,6.75,2.44%,2.17%,6.26%
Maximum,184.87,37.33,10.68%,8.93%,11.88%
Minimum,1.89,0.92,0.97%,0.00%,2.00%


In [20]:
results_dimensions[5][40]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,132,5.82,138,2.15,147,4.35%,6.12%,10.20%
1,133,21.97,137,4.56,144,2.92%,4.86%,7.64%
2,133,114.83,135,0.55,142,1.48%,4.93%,6.34%
3,132,13.39,135,13.28,135,2.22%,0.00%,2.22%
4,125,12.03,128,1.40,133,2.34%,3.76%,6.02%
5,130,11.09,132,11.57,135,1.52%,2.22%,3.70%
6,134,266.79,138,6.92,145,2.90%,4.83%,7.59%
7,129,33.14,133,7.74,138,3.01%,3.62%,6.52%
8,129,62.77,130,1.33,134,0.77%,2.99%,3.73%


In [21]:
results_dimensions[5][40]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,69.25,5.47,2.68%,3.76%,6.33%
Median,27.56,4.87,2.62%,4.00%,6.43%
Maximum,266.79,13.28,5.31%,6.12%,10.20%
Minimum,5.82,0.55,0.77%,0.00%,2.22%


In [22]:
results_dimensions[5][50]["granular_presentation"]

,Best VNS,Runtime VNS (s),Best Gurobi,Runtime Gurobi (s),Upper Bound,Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Instance,,,,,,,,
0,161,97.75,170,19.50,183,5.29%,7.10%,12.02%
1,143,22.80,151,40.58,170,5.30%,11.18%,15.88%
2,169,52.35,170,4.24,184,0.59%,7.61%,8.15%
3,166,27.42,169,8.82,178,1.78%,5.06%,6.74%
4,162,169.56,167,45.46,175,2.99%,4.57%,7.43%
5,158,48.67,162,16.03,171,2.47%,5.26%,7.60%
6,145,15.97,152,11.93,167,4.61%,8.98%,13.17%
7,144,266.33,147,14.36,150,2.04%,2.00%,4.00%
8,168,23.62,170,3.21,175,1.18%,2.86%,4.00%


In [23]:
results_dimensions[5][50]["summary_presentation"]

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,74.46,16.75,2.70%,6.91%,9.41%
Median,38.05,13.14,2.25%,6.18%,7.88%
Maximum,266.33,45.46,5.30%,14.47%,15.88%
Minimum,15.97,3.21,0.59%,2.00%,4.00%


In [24]:
results_all_instances = pd.DataFrame.from_dict(
    {
        instance: {
            "Best VNS": (best_vns := int((solution := best_solution_whithin_time_limit(solutions))["obj"])),
            "Runtime VNS (s)": solution["runtime"],
            "Best Gurobi": (
                best_gurobi := round_minor_deviations((gurobi_solutions := gurobi_60s[instance])[-1]["incumbent_obj"])
            ),
            "Runtime Gurobi (s)": runtime_best_objective_gurobi(gurobi_solutions, best_gurobi),
            "Upper Bound": (upper_bound_gurobi := round_minor_deviations(gurobi_solutions[-1]["obj_bound"])),
            "Gap VNS to Gurobi": (best_gurobi - best_vns) / best_gurobi,
            "Gap Gurobi": (upper_bound_gurobi - best_gurobi) / upper_bound_gurobi,
            "Gap VNS": (upper_bound_gurobi - best_vns) / upper_bound_gurobi,
        }
        for instance, solutions in vns_300s.items()
    },
    orient="index",
)

means = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].mean()

medians = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].median()

maxima = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].max()

minima = results_all_instances[
    ["Runtime VNS (s)", "Runtime Gurobi (s)", "Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]
].min()

summary_stats_all_instances = pd.DataFrame(
    [means, medians, maxima, minima], index=["Mean", "Median", "Maximum", "Minimum"]
)

summary_stats_all_instances_presentation = summary_stats_all_instances.copy()
summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]] = (
    summary_stats_all_instances_presentation[["Gap VNS to Gurobi", "Gap Gurobi", "Gap VNS"]].map("{:.2%}".format)
)
summary_stats_all_instances_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]] = (
    summary_stats_all_instances_presentation[["Runtime VNS (s)", "Runtime Gurobi (s)"]].map("{:.2f}".format)
)

In [25]:
summary_stats_all_instances_presentation

,Runtime VNS (s),Runtime Gurobi (s),Gap VNS to Gurobi,Gap Gurobi,Gap VNS
Mean,79.48,8.10,2.49%,4.12%,6.51%
Median,51.64,4.51,2.29%,4.33%,6.50%
Maximum,290.29,52.42,10.68%,14.47%,15.88%
Minimum,0.61,0.31,-0.98%,0.00%,0.00%


In [27]:
summary_stats_all_instances_presentation.to_csv("summary_all_instances.csv", sep=";")